In [2]:
from keras.layers import Input,Conv2D,Dense,Conv2DTranspose, Reshape,Lambda,LeakyReLU,Flatten,Dropout,Activation,UpSampling2D
from keras.layers.merge import _Merge
from keras.models import Model
from keras import backend as K
from keras.optimizers import Adam,RMSprop
from keras.activations import tanh

import numpy as np


Using TensorFlow backend.


In [1]:
class GAN():
    def __init__(
    self, input_dims,
        discriminator_conv_filters,
        discriminator_conv_kernel_size,
        discriminator_conv_strides,
        discriminator_activation,
        discriminator_dropout,
        discriminator_lr,
        generator_initial_dense_layer_size,
        generator_upsample,
        generator_conv_filters,
        generator_conv_kernel_size,
        generator_conv_strides,
        generator_activation,
        generator_dropout,
        generator_lr,
        optimizier,
        latent_dims
        
    ):
        
        self.input_dims=input_dims
        self.discriminator_conv_filters=discriminator_conv_filters
        self.discriminator_conv_kernel_size=discriminator_conv_kernel_size
        self.discriminator_conv_strides=discriminator_conv_strides
        self.discriminator_activation=discriminator_activation
        self.discriminator_dropout=discriminator_dropout
        self.discriminator_lr=discriminator_lr
        
        self.generator_initial_dense_layer_size=generator_initial_dense_layer_size
        self.generator_unsample=generator_upsample
        self.generator_conv_filters=generator_conv_filters
        self.generator_conv_kernel_size=generator_conv_kernel_size
        self.generator_conv_strides=generator_conv_strides
        self.generator_activation=generator_activation
        self.generator_dropout=generator_dropout
        self.generator_lr=generator_lr
        self.optimizier=optimizier
        self.latent_dims=latent_dims
        
        self.no_layers_discriminator=len(discriminator_conv_filters)
        self.no_layers_generator=len(generator_conv_filters)
        
        

        self.d_loss=[]
        self.g_loss=[]
        
        self._build_discriminator()
        self._build_generator()
        
        
    def get_activation(self,activation):
        if activation =='leaky_relu':
            layer=LeakyRelu(alpha=0.2)
        else:
            layer=Activation(activation) 
        return layer
    
    def _build_discriminator(self):
        discriminator_input=Input(shape=self.input_dims,name='discriminator_input')
        
        x=discriminator_input # I always forget this...everytime, and then have to deal with errors I've never even heard of...I dont know why this happens everytime.
        
        for i in range(self.no_layers_discriminator):
            x=Conv2D(
                filters=self.discriminator_conv_filters[i],
                kernel_size=self.discriminator_conv_kernel_size[i],
                strides=self.discriminator_conv_strides[i],
                padding='same',
                name='discriminator_layer_'+str(i),

                
            )(x)
            
            x=self.get_activation(self.discriminator_activation)(x)
            if self.discriminator_dropout:
                x=Dropout(rate=self.discriminator_dropout)(x)
            x=Flatten(x)
            
            discriminator_output=Dense(1,activation='sigmoid')(x)
            
            self.discriminator=Model(discriminator_input,discriminator_output)
            
            
            
        def _build_generator(self): #tricky part
            # mind the layers
            generator_input=Input(shape=(self.latent_dims,),name='generator_input')
            #input done
            # latent dims -> dense (shape maybe 7*7*64 ?) or np.prod(dense) ? -> reshape? ->connect to generator then...
            #input (latent dim size) to dense of number given in input ???
            # give the size of dense of generator as ip
            # make dense of np.prod(ip)
            # connect generator_input to this np.prod(dense) layer
            # now try -
            #  -- connect this to a smaller dense ?
            #  -- connect this to reshape?
            #  -- connect directly to unsample ?
             
                
            x= generator_input # dont forget this as I do always
            
            # trying small dens first, checking working
            #x=Dense(512)(x)
            
            # trying reshape to ip size specified
            x=Dense(np.prod(self.generator_initial_dense_layer_size))(x)
            x=self.get_activation(self.generator_activation)(x)
            
            x=Reshape(generator_initial_dense_layer_size)(x)
            
            if self.generator_dropout:
                x=Dropout(self.generator_dropout)(x)
            # reshape to prod of dims
            # connect to dense ? or connect to unsample ?
            
            
            for  i in range(self.no_layers_generator):
                if self.generator_upsample == 2:
                    x=UpSampling2D()(x)
                    x=Conv2D(
                        filters=self.generator_conv_filters[i],
                        kernel_size=self.generator_conv_kernel_size[i],
                        strides=self.generator_conv_strides[i],
                        padding='same',
                        name='generator_layer_'+str(i)
                    
                    )(x)
                else:
                    Conv2DTranspose(
                        filters=self.generator_conv_filters[i],
                        kernel_size=self.generator_conv_kernel_size[i],
                        strides=self.generator_conv_strides[i],
                        padding='same',
                        name='generator_layer_'+str(i)                        
                    
                    )(x)
                    
            if i<self.no_layers_generator-1:
                x=self.get_activation(self.generator_activation)(x)
            
            else:
                x=Activation('tanh')(x)
                
            generator_output=x
            
            
            self.generator=Model(generator_input,generator_output)
        
        
        
        def get_opti(self, lr):
            if self.optimizer=='adam':
                opti=Adam(lr=lr,beta=0.5)
            elif self.optiimzer=='rmsprop':
                opti=RMSprop(lr=lr)
            else:
                opti=Adam(lr=lr)
            return opti
        
        
        def set_trainable(self,model_name,bool_value):
            '''
            for freezing dicrimimnaor training when required
            '''
            model_name.trainable=bool_value
            for l in model_name.layers:
                l.trainable=bool_value
                
        def _build_adversial(self):
            # compile discriminator
            self.discriminator.compile(
            optimizer=self.get_opti(self.discriminator_lr),
                loss='binary_crossentropy',
                metrics=['accuracy']
            )
            
            self.set_trainable(self.discriminator,False)
            
            model_input=Input(shape=(self.latent_dims,),name='model_input')
            model_output=self.discriminator(self.generator(model_input))
            self.model=Model(model_input,model_output)
            
            
            self.model.compile(optimizer=self.get_opti(self.discriminator_lr),
                              loss='binary_crossentropy',
                              metrics=['accuracy'])
            
            self.set_trainable(self.discriminator,True)
            
            
        def train_discrimminator(self,x_train,batch_size,using_generator):
            valid=np.ones((batch_size,1))
            fake=np.zeros((batch_size,1))
            
            #train on real images
            if using_generator:
                true_imgs=next(x_train)[0]
                if true_imgs.shape[0]!=batch_size:
                    true_imgs=next(x_train)[0]
            else:
                idx=np.random.randn(0,x_train.shape[0],batch_size)
                true_imgs=x_train[idx]
                
            #train on generated images
            noise=np.random.normal(0,1,(batch_size,self.latent_dims))
            
            gen_images=self.generator.predict(noise)
            
            d_loss_real,d_acc_real=self.discriminator.train_on_batch(true_imgs,valid)
            d_loss_fake,d_acc_fake=self.discriminator.train_on_batch(gen_imgs,fake)
            
            d_loss=0.5*(d_loss_real+d_loss_fake) #avg of real n fake
            d_acc=0.5*(d_acc_real+d_acc_fake)
            return [d_loss,d_loss_real,d_loss_fake,d_acc,d_acc_real,d_acc_fake]
        
        
        def train_generator(self,batch_size):
            valid=np.ones((batch_size,1))
            noise=np.random.normal(0,1,(batch_size,latent_dims))
            return self.model.train_on_batch(noise,valid)
        
        
        def train(self,x_train,batch_size,epochs,run_folder,print_every_n_batch=50,using_generator=False):
            for epoch in range(self.epoch,slf.epoch+epochs):
                d=self.train_discriminator(x_train,batch_size,using_generator)
                g=self.train_generator(batch_size)
                
                print ("%d [D loss: (%.3f)(R %.3f, F %.3f)] [D acc: (%.3f)(%.3f, %.3f)] [G loss: %.3f] [G acc: %.3f]" % (epoch, d[0], d[1], d[2], d[3], d[4], d[5], g[0], g[1]))
                self.d_losses.append(d)
                self.g_losses.append(g)
                
                
                if epoch % print_every_n_batch==0:
                    self.sample_images(run_folder)
                    self.model.save_weights(os.path.join(run_folder,'weights/weights-%d.h5'%(epoch)))
                    self.model.save_weights(os.path.join(run_folder,'weights/weights.h5'))
                    self.save_model(run_folder)
                self.epoch+=1
            
            
        def save(self,folder):
            with open(os.path.join(folder,'params.pkl'),'wb')as f:
                pkl.dump([
                    self.latent_dims,
                    self.discriminator_conv_filters,
                    self.discriminator_conv_kernel_size,
                    self.discriminator_conv_strides,
                    self.discriminator_activation,
                    self.discriminator_dorpout,
                    self.discriminator_initial_dense_layer_size,
                    self.discriminator_lr,
                    self.discriminator_input,
                    self.discriminator_output,
                ])
                
